## __LLM__

We will be using HuggingFace llm

In [ ]:
# import for the llm
from langchain_community.llms import HuggingFaceEndpoint


In [ ]:
# llm
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
hf_token = "[YOUR_HUGGING_FACE_API_KEY]"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, huggingfacehub_api_token=hf_token
)

## __Tool__

Google Serper SEARCH for searching on internet

In [ ]:
# import
from crewai_tools import SerperDevTool


In [ ]:
# import
from crewai_tools import SerperDevTool


Tool for web scrapping

In [ ]:
# import
from crewai_tools import ScrapeWebsiteTool

In [ ]:
# instancing
scrap_tool = ScrapeWebsiteTool()

__Agent__

In [ ]:
# import
from crewai import Agent

In [ ]:
# find a dish
dish_search_hunter = Agent(
    role = "Dish Searcher",
    goal = "Your goal is to find the best dish that can be made from {provided items} using the search tool provided to you. Return the webpage url you have found.",
    backstory = "You are a headchef in a multicusine restaurant. You own a 3 michelin star restaurant.You are a professional chef.",
    tool = search_tool,
    llm = llm,
    allow_delegation = False,

)

In [ ]:
# content writer
content_writer = Agent(
    role = "Content Writer",
    goal = "you write a content from the information from a webpage in simple words",
    backstory = " you are good at comprehencing the content of a webpage and re-writing it in simple words",
    llm = llm,
    allow_delegation = False,
    max_iter = 5,
    tool = scrap_tool
)

In [ ]:
# step maker
step_maker = Agent(
    role = "Step writer",
    goal = "You prepare a to-do list from a provided content according to the workflow to achieve the final result",
    backstory = "You prepare very good to-do lists from a content",
    llm = llm,
    allow_delegation = False,
    max_iter = 5
)

__Task__

In [ ]:
# import
from crewai import Task

In [ ]:
# dish finding
dish_search_task = Task(
    description = """
    Search in internet to get the best dish possible from {provided items}.
    select the first result.
    return the web-page url""",
    expected_output = "an url",
    agent = dish_search_hunter,
    tool = search_tool.run,
    async_execution = False
)

In [ ]:
# content write
content_write_task = Task(
    description = "Read the content of the webpage. Write the content of the page in simple words",
    expected_output = "a paragraph in simple words",
    agent = content_writer,
    async_execution = False,
    tool = scrap_tool
)

In [ ]:
# step write
step_writer_task = Task(
    description = "rewrite the content in the form of to-do list in such a way that, following the to-do list from top to bottom will result the final product. Make sure you add the title of the content at the top of todo list as heading.",
    expected_output = "a to-do list",
    agent = step_maker,
    async_execution = False,
    output_file = "todo.md"
)

## __Crew__

Creating a crew object to run it

In [ ]:
# import
from crewai import Crew, Process

In [ ]:
crew = Crew(
    tasks = [dish_search_task,content_write_task,step_writer_task],
    agents = [dish_search_hunter,content_writer, step_maker],
    process = Process.sequential,

    verbose = True
)

In [ ]:
def run_crew(ingredients : str):
  crew.kickoff(inputs = {"provided items" : ingredients})

In [ ]:
result = run_crew(str(input("Write the available ingredients you have")))